<a href="https://colab.research.google.com/github/divyalaldinani/Deep-Learning-in-TensorFlow/blob/main/05_transfer_learning_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [7]:
!wget https://github.com/divyalaldinani/Deep-Learning-in-TensorFlow/blob/main/extras/helper_functions.py

--2025-01-14 16:06:01--  https://github.com/divyalaldinani/Deep-Learning-in-TensorFlow/blob/main/extras/helper_functions.py
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘helper_functions.py.1’

helper_functions.py     [  <=>               ] 281.16K   852KB/s    in 0.3s    

2025-01-14 16:06:02 (852 KB/s) - ‘helper_functions.py.1’ saved [287912]



In [10]:
from helper_functions import pred_and_plot, plot_loss_curves, unzip_folder, walk_through_directory

### Downloading the dataset

In [12]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_folder("10_food_classes_10_percent.zip")

--2025-01-14 16:08:46--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.137.207, 142.250.101.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.137.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   258MB/s    in 0.6s    

2025-01-14 16:08:47 (258 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [14]:
walk_through_directory('/content/10_food_classes_10_percent')

2 directories and 0 files in /content/10_food_classes_10_percent
10 directories and 0 files in /content/10_food_classes_10_percent/test
0 directories and 250 files in /content/10_food_classes_10_percent/test/chicken_curry
0 directories and 250 files in /content/10_food_classes_10_percent/test/ice_cream
0 directories and 250 files in /content/10_food_classes_10_percent/test/steak
0 directories and 250 files in /content/10_food_classes_10_percent/test/hamburger
0 directories and 250 files in /content/10_food_classes_10_percent/test/grilled_salmon
0 directories and 250 files in /content/10_food_classes_10_percent/test/sushi
0 directories and 250 files in /content/10_food_classes_10_percent/test/pizza
0 directories and 250 files in /content/10_food_classes_10_percent/test/ramen
0 directories and 250 files in /content/10_food_classes_10_percent/test/fried_rice
0 directories and 250 files in /content/10_food_classes_10_percent/test/chicken_wings
10 directories and 0 files in /content/10_food

In [15]:
train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

#### Input Data Pipelines
**using tf.keras.preprocessing.image_dataset_from_directory() -> creates tf.data.Dataset object, explore [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory)**

In [19]:
import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

BATCH_SIZE= 32
IMAGE_SIZE = (224, 224)

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                         batch_size=BATCH_SIZE,
                                                                         image_size=IMAGE_SIZE,
                                                                         shuffle=True,
                                                                         label_mode='categorical' # one hot encoded
                                                                         )

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                             batch_size=BATCH_SIZE,
                                                             image_size = IMAGE_SIZE,
                                                             shuffle=True,
                                                             label_mode='categorical')

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [20]:
train_data_10_percent

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [21]:
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [23]:
# see a batch of Data

for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[8.99799118e+01 8.25637741e+01 1.01812180e+02]
   [7.47889023e+01 6.79008331e+01 8.58220673e+01]
   [6.60758896e+01 6.18223839e+01 7.72110901e+01]
   ...
   [1.33571424e+01 1.23571424e+01 1.73571434e+01]
   [1.33571424e+01 1.23571424e+01 1.73571434e+01]
   [1.32311926e+01 1.42311926e+01 1.82311916e+01]]

  [[7.46125641e+01 7.08686218e+01 9.30918350e+01]
   [8.22959213e+01 7.92691345e+01 1.00023598e+02]
   [7.16910095e+01 7.05481567e+01 8.84767227e+01]
   ...
   [1.30000000e+01 1.20000000e+01 1.70000000e+01]
   [1.30000000e+01 1.20000000e+01 1.70000000e+01]
   [1.26011009e+01 1.36011009e+01 1.76011009e+01]]

  [[8.86648636e+01 8.96600723e+01 1.15735970e+02]
   [8.37662659e+01 8.61948395e+01 1.11532211e+02]
   [8.56744232e+01 8.81029968e+01 1.11960136e+02]
   ...
   [1.38364210e+01 1.28364210e+01 1.78364220e+01]
   [1.40000000e+01 1.30000000e+01 1.80000000e+01]
   [1.37857141e+01 1.47857141e+01 1.87857132e+01]]

  ...

  [[1.90755653e+01 1.20755663e+01 4.07556629e+00]
   [1

### Creating model with `tf.keras.applications`

In [29]:
# from tf.keras.applications import MobileV3Small

base_model = tf.keras.applications.MobileNetV3Small(
    weights='imagenet',
    input_shape=IMAGE_SIZE+(3,),
    include_top=False,
    include_preprocessing = True
)

base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name = 'input_layer')

x = base_model(inputs)

x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x) #returns (batch_size, channels)

outputs = tf.keras.layers.Dense(10, activation = 'softmax', name = 'output_layer')(x) # trains on batch_size requires (batch_size, features)

model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

history_0 = model_0.fit(x = train_data_10_percent,
                        validation_data=test_data,
                        epochs=5)

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.1223 - loss: 2.7049 - val_accuracy: 0.2796 - val_loss: 2.0568
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.3685 - loss: 1.9162 - val_accuracy: 0.5164 - val_loss: 1.5434
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.5787 - loss: 1.4092 - val_accuracy: 0.6356 - val_loss: 1.2568
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.6948 - loss: 1.1362 - val_accuracy: 0.6988 - val_loss: 1.0817
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.7510 - loss: 0.9366 - val_accuracy: 0.7360 - val_loss: 0.9677
